In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap
from my_tools import file_dic, plot_dic
import matplotlib.ticker as plticker
import string


In [3]:
path = '/media/onno/Algemeen/Thesis/'
path_to_save = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/forecast/'

In [4]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +file_dic['envelope']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})


ERA5RF_env_gen = xr.open_dataset(path+file_dic['envelope']['ERA5RF'],decode_times=False).squeeze()
GFS_env_gen = xr.open_dataset(path+file_dic['envelope']['GFS'],decode_times=False).squeeze()


GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
ERA5RF_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_env_gen.time.values]


GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_env_gen = ERA5RF_env_gen.assign_coords(lon=(((ERA5RF_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))


In [9]:
coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
(66,58,6,14)] #Norway/Sweden
# (60,52,46,54)] #Russia Samara/Kazan region
#list good and bad forecasts in all 8 regions
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
#list event types
eventz = ['persistent_hw','persistent_cw']

#list seasons
seasonz = ['JJA','DJF']
#list models and RWP envelope forecast data
modelz = ['GFS','ERA5RF']
modelz_labelz = ['GEFSRF','ERA5RF']
modelz_data = [GFS_env_gen,ERA5RF_env_gen]
#set good or bad forecast
qualityz = ['good','bad']
# #set contour 
# contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
#                      'persistent_cw': np.arange(24,61,2),
#                      'short_hw':np.arange(24,61,2),
#                      'short_cw':np.arange(24,61,2)},
#                'JJA':{'persistent_hw':np.arange(16,61,2),
#                      'persistent_cw': np.arange(16,61,2),
#                      'short_hw':np.arange(16,61,2),
#                      'short_cw':np.arange(16,61,2)}}

#set lead day
lead_day = 5
#Define letters for subtiles
letterz = np.array(['a','c','b','d']).reshape(2,2)
#create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(10,8))
#loop over models
for i,model in enumerate(modelz):
    #create empty list to append data to for all locations and concatenate data
    fcst_data = modelz_data[i]
    #loop over good and bad forecasts
    for j,quality in enumerate(qualityz):
        fcst_env_quality = []
        rean_env_quality = []
        #loop over all locations
        for lat_0,lat_1,lon_0,lon_1 in coordinatez:
            #set coordinates to average over sepcific latitudes and all longitudes. Latitude represents 24 degrees
            lat_min = lat_1 - 8
            lat_max = lat_0 + 6
            #create empty list to append data to for all events and concatenate data
            fcst_env_loc = []
            rean_env_loc = []
            #loop over events
            for event in eventz:
                #create empty list to append data to for all seasons and concatenate data
                fcst_env_event = []
                rean_env_event = []
                #loop over events
                for season in seasonz:
                    #load good and bad forecasts and convert index to datetime
                    df_quality = pd.read_csv(path+file.format(quality,event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                    df_quality.index = pd.to_datetime(df_quality.index)
                    #create empty list to append data to for all good or bad forecasts and concatenate data
                    fcstz_env_season = []
                    reanz_env_season = []
                    #loop over all dates in good or bad forecasts
                    for date in df_quality.index:
                        if lon_0>180:
                            lon_0_calc = lon_0 - 360
                        else:
                            lon_0_calc = lon_0 
                        #select data on 5 days before event onset and 10 days after first selected date
                        time_begin = date - pd.Timedelta(lead_day,'days')
                        time_end = time_begin + pd.Timedelta(10,'days')
                        #select all relevant data 8 degrees south and 8 degrees north of box
                        rean_env_quality_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                       time=slice(time_begin,time_end))
                        rean_env_quality_sub = rean_env_quality_sub.assign_coords(lon=(((rean_env_quality_sub.lon-lon_0_calc+180)%360)-180)).sortby('lon')
                        #for ERA5RF select only data in 12 hour interval for fair comparison
                        if model=='ERA5RF':
                            rean_env_quality_sub = rean_env_quality_sub.sel(time = rean_env_quality_sub.time.values[[bool(i) for i in rean_env_quality_sub.time.dt.hour%12==0]])
                        #change time coordinates from datetime to days before/after onset
                        rean_env_quality_sub = rean_env_quality_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_quality_sub.time)))
                        #select forecast data. Take al forecasts between 0 and 240 hours and all latitudes 8 degrees south and 8 degrees north of box
                        fcst_env_quality_sub = fcst_data.sel(lat=slice(lat_max,lat_min),time=time_begin,lead=slice(0,240))
                        fcst_env_quality_sub = fcst_env_quality_sub.assign_coords(lon=(((fcst_env_quality_sub.lon-lon_0_calc+180)%360)-180)).sortby('lon')
                        fcst_env_quality_sub = fcst_env_quality_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_env_quality_sub.time)))

                        reanz_env_season.append(rean_env_quality_sub)
                        fcstz_env_season.append(fcst_env_quality_sub)
                    rean_env_season_nc = xr.concat(reanz_env_season,dim='time').groupby('time').mean().mean('lat')
                    fcst_env_season_nc = xr.concat(fcstz_env_season,dim='lead').groupby('lead').mean().mean('lat')
                    rean_env_event.append(rean_env_season_nc)
                    fcst_env_event.append(fcst_env_season_nc)
                fcst_env_event_nc = xr.concat(fcst_env_event,dim='season').mean('season')
                rean_env_event_nc = xr.concat(rean_env_event,dim='season').mean('season')
                fcst_env_loc.append(fcst_env_event_nc)
                rean_env_loc.append(rean_env_event_nc)
            fcst_env_loc_nc = xr.concat(fcst_env_loc,dim='event').mean('event')
            rean_env_loc_nc = xr.concat(rean_env_loc,dim='event').mean('event')
            fcst_env_quality.append(fcst_env_loc_nc)
            rean_env_quality.append(rean_env_loc_nc)
        fcst_env_quality_nc = xr.concat(fcst_env_quality,dim='loc').mean('loc')
        rean_env_quality_nc = xr.concat(rean_env_quality,dim='loc').mean('loc') 
        fcst_env_quality_nc.to_netcdf(path_to_save+'fcst_hovmoller_persistent_extremes_NW_Europe.nc')
        rean_env_quality_nc.to_netcdf(path_to_save+'rean_hovmoller_persistent_extremes_NW_Europe.nc')

        ax = axz[j,i]
        ax.text(x=0,y=1.1,s=letterz[i,j]+')',transform=ax.transAxes,fontweight='bold',fontsize=17,
                   verticalalignment='top')
        lonz = rean_env_quality_nc.lon.values
        dayz = rean_env_quality_nc.time.values

        clr_levels_contourf = np.arange(10,25.01,.1)
        clr_levels_contour = np.arange(10,25.01,2)
        cs = ax.contour(lonz,dayz,rean_env_quality_nc.v.values,colors='k',levels=clr_levels_contour,extend='both',linewidths=1)
        im = ax.contourf(lonz,dayz,fcst_env_quality_nc.v - rean_env_quality_nc.v.values,cmap='bwr',
                         extend='both',levels=np.linspace(-3,3,25),alpha=.5,antialiased=True)
        
        ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
        
        if j==0:
            ax.set_title('{} \n{} Forecasts'.format(modelz_labelz[i],quality.title()),
                        fontsize=15)
        else:
            ax.set_title('{} Forecasts'.format(quality.title()),
                        fontsize=15)
            ax.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax.axvline(x=6,color='grey')
        ax.axhline(y=0,color='grey')
        ax.set_yticks(np.linspace(-5,5,5))
        if i==0:
            ax.set_ylabel('Days before/after onset event',fontsize=12)
        ax.set_xticks([-120,-60,0,60,120])
        ax.tick_params(labelsize=12)
        ax.label_outer()


# fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

fig.subplots_adjust(left=0.1,right=0.97,bottom=0.16,top=0.92,hspace=0.15,wspace=0.05)
cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
cbar.ax.set_xlabel('E (m/s)',fontsize=12)
cbar.ax.tick_params(labelsize=12)
fig.savefig(path+'/Main_Figures/hovmoller_forecast_errors_NW_EUROPE',dpi=200)




